In [48]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [49]:
train = pd.read_csv('data/train.csv', index_col=0)
train.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [50]:
X = train.drop('target', axis=1)
y = train.target

In [94]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1500, n_iter=15, random_state=42)
svd.fit(X)
print(svd.explained_variance_ratio_.sum())

0.9952524035769564


In [95]:
from sklearn.model_selection import train_test_split

X_svd = np.abs(svd.transform(X))

X_added = pd.concat([X.reset_index().drop('ID', axis=1), pd.DataFrame(X_svd)], axis=1)

X_log = np.log1p(X_added)
y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(
    X_log, y_log, test_size=0.2, random_state=42
)

In [96]:
X_log.head()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,14.502851,11.708198,14.425517,14.499043,14.118673,13.984267,13.483200,13.953357,14.860196,12.038860
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.603968,0.0,...,13.397898,13.653013,13.607073,14.853827,15.186509,14.441952,8.263016,14.355081,11.186887,13.678884
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,12.631499,10.802247,12.670341,11.168139,12.551212,12.490028,11.483832,10.715498,12.455815,9.812598
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,9.703064,11.589393,12.289598,10.156686,12.222104,10.567928,12.159510,10.696191,11.993346,12.768694
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.508658,0.0,...,12.250407,12.877133,12.570345,11.146848,10.670044,13.417807,14.278171,13.457753,12.314243,13.170672


In [97]:
def rmsle_metric(y_test, y_pred): 
    assert len(y_test) == len(y_pred)
    y_test = np.exp(y_test)-1
    y_pred = np.exp(y_pred)-1
    rmsle = np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))
    return ('RMSLE', rmsle, False)

In [99]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=31,
    learning_rate=0.008,
    n_estimators=600
)

gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=rmsle_metric,
        early_stopping_rounds=100
)

[1]	valid_0's l2: 2.86638	valid_0's RMSLE: 1.69304
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l2: 2.85575	valid_0's RMSLE: 1.6899
[3]	valid_0's l2: 2.84521	valid_0's RMSLE: 1.68678
[4]	valid_0's l2: 2.83429	valid_0's RMSLE: 1.68353
[5]	valid_0's l2: 2.8242	valid_0's RMSLE: 1.68054
[6]	valid_0's l2: 2.8135	valid_0's RMSLE: 1.67735
[7]	valid_0's l2: 2.80275	valid_0's RMSLE: 1.67414
[8]	valid_0's l2: 2.79287	valid_0's RMSLE: 1.67119
[9]	valid_0's l2: 2.78222	valid_0's RMSLE: 1.668
[10]	valid_0's l2: 2.7727	valid_0's RMSLE: 1.66514
[11]	valid_0's l2: 2.76297	valid_0's RMSLE: 1.66222
[12]	valid_0's l2: 2.75382	valid_0's RMSLE: 1.65946
[13]	valid_0's l2: 2.74389	valid_0's RMSLE: 1.65647
[14]	valid_0's l2: 2.73546	valid_0's RMSLE: 1.65392
[15]	valid_0's l2: 2.72619	valid_0's RMSLE: 1.65112
[16]	valid_0's l2: 2.7164	valid_0's RMSLE: 1.64815
[17]	valid_0's l2: 2.7074	valid_0's RMSLE: 1.64542
[18]	valid_0's l2: 2.69824	valid_0's RMSLE: 1.64263
[19]	valid_0's l2:

[158]	valid_0's l2: 2.16378	valid_0's RMSLE: 1.47098
[159]	valid_0's l2: 2.16248	valid_0's RMSLE: 1.47054
[160]	valid_0's l2: 2.16081	valid_0's RMSLE: 1.46997
[161]	valid_0's l2: 2.15952	valid_0's RMSLE: 1.46953
[162]	valid_0's l2: 2.15729	valid_0's RMSLE: 1.46877
[163]	valid_0's l2: 2.1558	valid_0's RMSLE: 1.46826
[164]	valid_0's l2: 2.15376	valid_0's RMSLE: 1.46757
[165]	valid_0's l2: 2.15275	valid_0's RMSLE: 1.46723
[166]	valid_0's l2: 2.15068	valid_0's RMSLE: 1.46652
[167]	valid_0's l2: 2.14823	valid_0's RMSLE: 1.46568
[168]	valid_0's l2: 2.14701	valid_0's RMSLE: 1.46527
[169]	valid_0's l2: 2.14536	valid_0's RMSLE: 1.4647
[170]	valid_0's l2: 2.14394	valid_0's RMSLE: 1.46422
[171]	valid_0's l2: 2.1423	valid_0's RMSLE: 1.46366
[172]	valid_0's l2: 2.14119	valid_0's RMSLE: 1.46328
[173]	valid_0's l2: 2.13988	valid_0's RMSLE: 1.46283
[174]	valid_0's l2: 2.13872	valid_0's RMSLE: 1.46244
[175]	valid_0's l2: 2.13761	valid_0's RMSLE: 1.46206
[176]	valid_0's l2: 2.13614	valid_0's RMSLE: 1.46

[314]	valid_0's l2: 2.04845	valid_0's RMSLE: 1.43124
[315]	valid_0's l2: 2.04743	valid_0's RMSLE: 1.43088
[316]	valid_0's l2: 2.04781	valid_0's RMSLE: 1.43102
[317]	valid_0's l2: 2.04757	valid_0's RMSLE: 1.43093
[318]	valid_0's l2: 2.04669	valid_0's RMSLE: 1.43062
[319]	valid_0's l2: 2.04561	valid_0's RMSLE: 1.43025
[320]	valid_0's l2: 2.04591	valid_0's RMSLE: 1.43035
[321]	valid_0's l2: 2.04528	valid_0's RMSLE: 1.43013
[322]	valid_0's l2: 2.04431	valid_0's RMSLE: 1.42979
[323]	valid_0's l2: 2.04305	valid_0's RMSLE: 1.42935
[324]	valid_0's l2: 2.04301	valid_0's RMSLE: 1.42934
[325]	valid_0's l2: 2.04323	valid_0's RMSLE: 1.42942
[326]	valid_0's l2: 2.04247	valid_0's RMSLE: 1.42915
[327]	valid_0's l2: 2.04223	valid_0's RMSLE: 1.42907
[328]	valid_0's l2: 2.04178	valid_0's RMSLE: 1.42891
[329]	valid_0's l2: 2.04082	valid_0's RMSLE: 1.42857
[330]	valid_0's l2: 2.04012	valid_0's RMSLE: 1.42833
[331]	valid_0's l2: 2.03981	valid_0's RMSLE: 1.42822
[332]	valid_0's l2: 2.03882	valid_0's RMSLE: 1

[470]	valid_0's l2: 2.00677	valid_0's RMSLE: 1.4166
[471]	valid_0's l2: 2.00642	valid_0's RMSLE: 1.41648
[472]	valid_0's l2: 2.00627	valid_0's RMSLE: 1.41643
[473]	valid_0's l2: 2.00569	valid_0's RMSLE: 1.41622
[474]	valid_0's l2: 2.00535	valid_0's RMSLE: 1.4161
[475]	valid_0's l2: 2.00519	valid_0's RMSLE: 1.41605
[476]	valid_0's l2: 2.00498	valid_0's RMSLE: 1.41597
[477]	valid_0's l2: 2.00472	valid_0's RMSLE: 1.41588
[478]	valid_0's l2: 2.00456	valid_0's RMSLE: 1.41583
[479]	valid_0's l2: 2.00409	valid_0's RMSLE: 1.41566
[480]	valid_0's l2: 2.00422	valid_0's RMSLE: 1.4157
[481]	valid_0's l2: 2.00443	valid_0's RMSLE: 1.41578
[482]	valid_0's l2: 2.00415	valid_0's RMSLE: 1.41568
[483]	valid_0's l2: 2.00391	valid_0's RMSLE: 1.41559
[484]	valid_0's l2: 2.00424	valid_0's RMSLE: 1.41571
[485]	valid_0's l2: 2.00405	valid_0's RMSLE: 1.41565
[486]	valid_0's l2: 2.00374	valid_0's RMSLE: 1.41554
[487]	valid_0's l2: 2.00374	valid_0's RMSLE: 1.41553
[488]	valid_0's l2: 2.00305	valid_0's RMSLE: 1.41

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.008, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=600,
       n_jobs=-1, num_leaves=31, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [100]:
y_pred = gbm.predict(X_test)
print(rmsle_metric(y_test, y_pred))

('RMSLE', 1.412638864408138, False)
